In [1]:
# libraries and modules
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Embedding
from keras.layers import LSTM

from keras.optimizers import RMSprop
from keras.optimizers import SGD

from keras import utils as np_utils

import numpy as np
from PIL import Image
import glob
import os
import h5py

IMAGE_HEIGHT = 28
IMAGE_WIDTH = 28

Using TensorFlow backend.


In [2]:
# Accuracy Function

def accuracy(test_x, test_y, model):
    result = model.predict(test_x)
    predicted_class = np.argmax(result, axis=1)
    true_class = np.argmax(test_y, axis=1)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

In [3]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

In [5]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [6]:
# CNN Model 1

def CNN_1():
    
    model = Sequential()
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

# Add another convolutional layer

def CNN_2():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation="softmax"))
    
    return model

# Add Dropout

def CNN_3():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)))
    model.add(Conv2D(32, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation="softmax"))
    
    return model

In [7]:
# Compile Model 1

model1 = CNN_1()

model1.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])

# Compile Model 2

model2 = CNN_2()

model2.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])

# Compile Model 3

model3 = CNN_3()

model3.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])

In [8]:
# Model 1 Train
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 80s - loss: 0.3562 - acc: 0.9046 - val_loss: 0.1288 - val_acc: 0.9627
Epoch 2/10
 - 79s - loss: 0.1081 - acc: 0.9696 - val_loss: 0.0759 - val_acc: 0.9773
Epoch 3/10


KeyboardInterrupt: 

In [9]:
print(accuracy(X_test, y_test, model1))

98.71


In [40]:
# Model 3 Train
model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 277s - loss: 0.4237 - acc: 0.8670 - val_loss: 0.0757 - val_acc: 0.9774
Epoch 2/10
 - 263s - loss: 0.1636 - acc: 0.9527 - val_loss: 0.0521 - val_acc: 0.9816
Epoch 3/10


KeyboardInterrupt: 

In [9]:
print(accuracy(X_test, y_test, model3))

99.00999999999999


In [10]:
model3.save('mnistClassifier.h5')  # creates a HDF5 file

In [48]:
# Retrieves images from path

def get_images(path):
    
    images = []
    filenames = glob.glob(path)

    i = 0
    for f in filenames:
        image = Image.open(f).convert('L')
        image = image.resize((IMAGE_WIDTH,IMAGE_HEIGHT))
        images.append(np.array(image))
        i += 1

    print(i)
    images = np.array(images)
    print(images.shape)
    images = images.reshape(i, IMAGE_WIDTH, IMAGE_HEIGHT, 1)

    images = images.astype('float32')
    images /= 255
    
    return images

In [13]:
# Get labels for 2

path = "./digit_data"
NUM_CLASSES = 10

#new path of camera dir
parentDir = os.path.abspath(path)
new_dir = parentDir + "/images2/labels"

with open(new_dir) as f:
    labels_2 = f.readlines()

# remove whitespace characters like `\n` at the end of each line
labels_2 = [x.strip() for x in labels_2]

train_labels_2 = np.array(labels_2)
train_labels_2 = np_utils.to_categorical(train_labels_2, NUM_CLASSES)

print(train_labels_2)

# Get images
images_2 = get_images('./digits/images2/*.png')

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
10160
(10160, 28, 28)


In [14]:
# Accuracy of non-MNIST dataset

print(accuracy(images_2, train_labels_2, model3))

9.734251968503937


In [15]:
score = model3.evaluate(images_2, train_labels_2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 8.22564170492
Test accuracy: 0.097342519685


In [24]:
# Retrieves images from path
# Inverts images in black and white

def get_images_inverted(path):
    
    images = []
    filenames = glob.glob(path)

    i = 0
    for f in filenames:
        image = Image.open(f).convert('L')
        image = image.resize((IMAGE_WIDTH,IMAGE_HEIGHT))
        images.append(np.array(image))
        i += 1

    print(i)
    images = np.array(images)
    print(images.shape)
    images = images.reshape(i, IMAGE_WIDTH, IMAGE_HEIGHT, 1)

    images = images.astype('float32')
    images /= 255
    
    images = 1 - images
    
    return images

In [49]:
#Invert the image because model was trained in white background with black digits

#Test image is from hockey scoreboard, where the background is black and the digit is white
zero = get_images('./digit_data/0/*.jpg')

176
(176, 28, 28)


In [50]:
p = model3.predict(zero, batch_size=32, verbose=1)

176/176 [==============================] - 0s 2ms/step


In [51]:
print(p)

[[  9.92105808e-04   6.98830845e-05   8.35189785e-05 ...,   1.60068218e-02
    3.39768507e-04   9.78926361e-01]
 [  3.22699244e-03   3.50150949e-04   2.59005959e-04 ...,   1.79794058e-02
    7.10748543e-04   9.67356741e-01]
 [  1.76685214e-01   6.83412980e-03   7.02841282e-02 ...,   1.21143445e-01
    6.25763601e-03   3.95483106e-01]
 ..., 
 [  9.97914493e-01   3.12768123e-08   2.91453962e-05 ...,   2.13054427e-05
    5.35588333e-06   1.87039823e-05]
 [  9.98471916e-01   6.60541701e-08   3.06019465e-05 ...,   6.21182626e-05
    7.56929239e-06   4.27768864e-05]
 [  9.99828339e-01   4.38188792e-08   4.76524838e-06 ...,   1.14821623e-05
    1.86271718e-05   1.72896143e-05]]


In [52]:
print(np.argmax(p,1))

[9 9 9 0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 0 0 0 0 9 0
 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 6 0 0 0 9 0 0 0 6 0 0 0 0 0 0 9 6 0 0 0 0
 0 0 6 0 0 9 0 0 0 0 6 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 0 0 0 0 0 9 0
 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0
 0 0 0 0 0 9 0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 0 0 0 0 0]


In [53]:
#Invert the image because model was trained in white background with black digits

#Test image is from hockey scoreboard, where the background is black and the digit is white
one = get_images('./digit_data/1/*.jpg')

176
(176, 28, 28)


In [54]:
p_1 = model3.predict(one, batch_size=32, verbose=1)

176/176 [==============================] - 0s 2ms/step


In [55]:
print(p_1)

[[ 0.04037897  0.09269956  0.15313569 ...,  0.11371574  0.06447284
   0.1135007 ]
 [ 0.03546046  0.09483983  0.10690622 ...,  0.03852637  0.10558296
   0.12389571]
 [ 0.03433665  0.02188913  0.2279204  ...,  0.00779748  0.09645531
   0.0360094 ]
 ..., 
 [ 0.00676308  0.18148786  0.11468335 ...,  0.58250123  0.02993724
   0.00590438]
 [ 0.00370861  0.31655335  0.06364731 ...,  0.49851999  0.014361
   0.00504355]
 [ 0.00139787  0.35825115  0.01933993 ...,  0.53113043  0.02169169
   0.01742107]]


In [56]:
print(np.argmax(p_1,1))

[3 6 6 7 7 1 4 1 7 7 7 7 4 2 4 4 7 7 7 7 4 4 4 7 3 7 7 7 4 4 4 7 7 7 7 1 4
 4 4 7 7 7 7 4 4 4 3 9 9 9 9 9 9 9 9 9 9 6 9 9 9 9 9 9 9 9 9 9 6 9 9 9 9 9
 9 9 9 9 9 6 9 9 9 9 9 2 3 6 1 3 6 6 6 2 3 1 3 6 6 6 6 2 3 1 1 8 8 3 1 2 1
 1 1 8 8 3 1 1 1 1 3 8 8 3 1 1 1 1 8 8 3 1 1 1 1 1 8 8 3 1 1 1 3 7 7 7 7 1
 4 1 7 7 7 6 7 1 4 1 7 7 7 7 1 4 6 1 7 7 7 7 1 4 1 7 7 7]


In [57]:
#Invert the image because model was trained in white background with black digits

#Test image is from hockey scoreboard, where the background is black and the digit is white
two = get_images('./digit_data/2/*.jpg')

176
(176, 28, 28)


In [58]:
p_2 = model3.predict(two, batch_size=32, verbose=1)

176/176 [==============================] - 0s 2ms/step


In [59]:
print(np.argmax(p_2,1))

[3 2 2 2 2 2 2 2 0 0 0 0 0 2 7 7 0 0 0 0 0 7 7 0 2 0 0 0 0 7 7 0 0 0 0 4 0
 7 7 0 0 0 0 0 7 7 3 0 0 5 0 0 0 0 0 0 5 2 0 0 0 0 0 0 5 0 0 0 2 0 0 0 5 0
 0 0 0 0 0 2 5 0 0 0 0 2 2 2 4 3 2 2 2 2 2 4 3 2 2 2 2 2 2 4 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [60]:
three = get_images('./digit_data/3/*.jpg')
p_3 = model3.predict(three, batch_size=32, verbose=1)
print(np.argmax(p_3,1))

176
(176, 28, 28)
176/176 [==============================] - 0s 2ms/step
[2 4 2 3 3 3 3 3 2 3 3 2 3 1 8 3 2 3 3 2 3 8 3 2 1 3 3 2 3 8 3 2 3 3 2 1 3
 8 3 2 3 3 2 3 8 3 2 0 0 0 0 2 2 2 0 0 0 4 0 2 2 2 0 0 0 0 2 2 1 2 0 0 0 0
 2 2 2 0 0 2 0 0 2 2 2 1 1 1 1 2 4 1 2 1 1 1 2 4 2 1 2 1 1 1 3 3 3 2 3 1 3
 2 3 3 3 2 3 3 2 3 1 3 3 2 3 3 2 3 3 3 2 1 3 3 2 3 3 3 2 3 3 2 2 3 3 3 3 3
 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3]


In [61]:
four = get_images('./digit_data/4/*.jpg')
p_4 = model3.predict(four, batch_size=32, verbose=1)
print(np.argmax(p_4,1))

176
(176, 28, 28)
176/176 [==============================] - 0s 2ms/step
[4 4 2 2 2 2 2 2 2 8 8 8 8 6 8 8 2 8 8 8 8 8 8 2 6 8 8 8 8 8 8 2 8 8 8 4 8
 8 8 2 8 8 8 8 8 8 4 0 5 5 1 0 9 9 0 5 5 4 1 0 9 9 0 5 5 1 0 9 2 9 0 5 5 1
 0 9 9 0 5 2 5 1 0 9 9 6 6 2 4 4 4 2 2 6 6 4 4 4 2 2 2 6 6 4 1 8 3 1 1 6 2
 6 1 8 3 1 1 2 6 1 6 8 3 1 1 2 6 1 8 3 1 4 1 2 6 1 8 3 1 1 2 6 4 2 2 2 2 2
 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [62]:
five = get_images('./digit_data/5/*.jpg')
p_5 = model3.predict(five, batch_size=32, verbose=1)
print(np.argmax(p_5,1))

176
(176, 28, 28)
176/176 [==============================] - 0s 2ms/step
[3 7 2 5 5 5 5 5 3 3 5 5 5 3 5 8 3 3 5 5 5 5 8 3 2 3 5 5 5 5 8 3 3 5 5 3 5
 5 8 3 3 5 5 5 5 8 3 8 8 2 2 2 8 8 8 8 2 7 2 2 8 8 8 8 2 2 2 8 2 8 8 8 2 2
 2 8 8 8 8 2 2 2 2 8 8 3 2 2 3 3 7 2 2 3 2 3 3 7 2 2 2 3 2 3 5 5 3 3 3 3 9
 5 5 5 3 3 3 9 5 5 2 5 3 3 3 9 5 5 5 3 3 3 3 9 5 5 5 3 3 3 9 5 3 5 5 5 5 5
 5 5 5 5 5 7 5 5 5 5 5 5 5 5 5 5 2 5 5 5 5 5 5 5 5 5 5 5]


In [63]:
six = get_images('./digit_data/6/*.jpg')
p_6 = model3.predict(six, batch_size=32, verbose=1)
print(np.argmax(p_6,1))

176
(176, 28, 28)
176/176 [==============================] - 0s 2ms/step
[2 2 2 6 0 6 6 6 8 8 8 6 8 2 8 8 8 8 8 6 8 8 8 8 1 8 8 6 8 8 8 8 8 8 6 9 8
 8 8 8 8 8 6 8 8 8 2 8 8 1 8 8 8 8 8 8 1 2 8 8 8 8 8 8 1 8 8 8 2 8 8 8 1 8
 8 8 8 8 8 2 1 8 8 8 8 2 1 2 9 2 2 2 2 2 1 9 2 2 2 2 2 2 1 9 9 8 3 3 3 2 9
 9 9 8 3 3 3 9 9 9 1 8 3 3 3 9 9 9 8 3 3 9 3 9 9 9 8 3 3 3 9 9 2 6 6 6 0 6
 6 6 6 6 6 2 0 6 6 6 6 6 6 0 6 6 2 6 6 6 6 0 6 6 6 6 6 0]


In [64]:
seven = get_images('./digit_data/7/*.jpg')
p_7 = model3.predict(seven, batch_size=32, verbose=1)
print(np.argmax(p_7,1))

176
(176, 28, 28)
176/176 [==============================] - 0s 2ms/step
[7 7 6 7 7 7 7 7 7 7 7 7 7 6 7 7 7 7 7 7 7 7 7 7 6 7 7 7 7 7 7 7 7 7 7 6 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 9 9 9 9 7 7 7 7 9 9 9 9 7 7 7 9 9 9 6 9 7 7 7 9
 9 9 9 7 7 6 7 9 9 9 9 6 6 6 6 7 7 6 6 6 6 6 7 7 6 6 6 6 6 6 2 2 2 0 0 6 2
 6 2 2 2 0 0 2 6 2 6 2 2 0 0 2 6 2 2 2 0 6 0 2 6 2 2 2 0 0 2 6 7 2 2 7 7 7
 7 7 2 2 7 7 7 7 7 7 2 2 7 7 7 7 6 7 2 2 7 7 7 7 7 2 2 7]


In [65]:
eight = get_images('./digit_data/8/*.jpg')
p_8 = model3.predict(eight, batch_size=32, verbose=1)
print(np.argmax(p_8,1))

176
(176, 28, 28)
176/176 [==============================] - 0s 2ms/step
[2 2 2 8 8 8 8 8 8 8 8 8 8 2 8 8 8 8 8 8 8 8 8 8 1 8 8 8 8 8 8 8 8 8 8 1 8
 8 8 8 8 8 8 8 8 8 2 8 1 1 6 6 1 8 8 1 1 2 6 6 1 8 8 1 1 6 6 1 2 8 8 1 1 6
 6 1 8 8 1 2 1 6 6 1 8 2 1 2 1 2 2 2 2 2 1 1 2 2 2 2 2 2 1 1 2 3 3 3 3 2 3
 3 2 3 3 3 3 3 3 2 1 3 3 3 3 3 3 2 3 3 3 1 3 3 3 2 3 3 3 3 3 3 2 8 8 8 8 8
 8 8 8 8 8 2 8 8 8 8 8 8 8 8 8 8 2 8 8 8 8 8 8 8 8 8 8 8]


In [66]:
nine = get_images('./digit_data/9/*.jpg')
p_9 = model3.predict(nine, batch_size=32, verbose=1)
print(np.argmax(p_9,1))

176
(176, 28, 28)
176/176 [==============================] - 0s 2ms/step
[4 1 2 8 9 8 9 8 0 0 0 0 0 3 5 0 0 0 0 0 0 5 0 0 3 0 0 0 0 5 0 0 0 0 0 7 0
 5 0 0 0 0 0 0 5 0 4 8 0 2 2 6 6 8 8 0 2 1 2 6 6 8 8 0 2 2 6 6 3 8 8 0 2 2
 6 6 8 8 0 2 2 2 6 6 8 3 3 3 7 4 1 3 2 3 3 7 4 1 2 3 2 3 3 7 9 9 9 2 9 3 9
 9 9 9 9 2 9 9 9 9 3 9 9 2 9 9 9 9 9 9 2 7 9 9 9 9 9 9 2 9 9 9 4 6 6 8 9 8
 9 8 6 6 8 1 9 8 9 8 6 6 8 9 8 9 3 8 6 6 8 9 8 9 8 6 6 9]
